In [8]:
%matplotlib inline

In [9]:
import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from datetime import datetime, date
import timeit
from collections import defaultdict, OrderedDict
import tabulate
import time
import glob
from functools import reduce

timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

matplotlib.style.use('ggplot')

RANDOM_SEED = 33

In [6]:
def shift_encrypt(plain, k, n=26):
    plain = plain.upper().replace(' ', '')
    plain = [ord(x) - ord('A') for x in plain]
    cipher = [(x + k) % n for x in plain]
    cipher = [chr(x + ord('A')) for x in cipher]
    return ''.join(cipher)

def shift_decrypt(cipher, k, n=26):
    return shift_encrypt(cipher, -k, n)

' '.join(shift_decrypt('ZLOOA WKLVA EHARQ WKHA ILQDO', 3).split('X'))

'WILL THIS BE ONTHE FINAL'

In [65]:
def affine_encrypt(plain, A, b, n=26):
    plain = plain.upper().replace(' ', '')
    if len(plain) % 2:
        plain = plain + 'X'
        
    plain = [ord(x) - ord('A') for x in plain]
    plain_vecs = [np.asarray(plain[i:i+2]).reshape((2, 1)) for i in range(0, len(plain), 2)]
    cipher = np.asarray([np.mod(A.dot(p) + b, n) for p in plain_vecs])
    cipher = cipher.flatten()
    cipher = [chr(x + ord('A')) for x in cipher]
    return ''.join(cipher)


A = np.matrix([[3, 4], [2, 3]])
b = np.matrix([[2], [5]])
cipher = affine_encrypt('CRYPTOLOGY', A, b)
print(cipher)

YIEULHNRML


In [59]:
A_inv = np.invert(A)
b_inv = A_inv.dot(b)

np.mod(A_inv.dot(np.matrix([[13], [5]])) - b_inv, 26)

matrix([[ 8],
        [19]])

In [57]:
ord('F') - ord('A')

5

In [67]:
A = np.matrix([[3, 5], [1, 2]])
b = np.matrix([[2], [2]])
cipher = affine_encrypt('HELP', A, b)
print(cipher)

RRGR


In [63]:
A_inv = np.mod(np.invert(A), 26)
A_inv

matrix([[22, 20],
        [24, 23]])

In [98]:
def powers_of_two_mod(base, exp, mod):
    current = base
    indices = [i for i, x in enumerate(bin(exp)[2:][::-1]) if x == '1']
    powers=' + '.join(['2^{ind}'.format(ind=ind) for ind in indices])
    print('{exp} = {powers}'.format(exp=exp, powers=powers))
    
    n = indices[-1]
    components = []
    if 0 in indices:
        print('{base}^(2^{i}) = {current} (mod {mod})'.format(base=base, i=0, current=current, mod=mod))
        components.append(current)
        
    for i in range(n):
        current = (current ** 2) % mod
        if i + 1 in indices: 
            print('{base}^(2^{i}) = {current} (mod {mod})'.format(base=base, i=i + 1, current=current, mod=mod))
            components.append(current)
    
    print('{base}^({exp}) (mod {mod}) = {base}^({powers}) (mod {mod})'.format(base=base, exp=exp, mod=mod,
                                                      powers=powers))
    
    print('= {prod} (mod {mod})'.format(base=base, exp=exp, mod=mod,
                                                      prod=' * '.join([str(x) for x in components])))
    prod = np.product(components)
    print('= {prod}(mod {mod}) = {result} (mod {mod})'.format(mod=mod, prod=prod, result=prod%mod))
        
powers_of_two_mod(271, 321, 481)

321 = 2^0 + 2^6 + 2^8
271^(2^0) = 271 (mod 481)
271^(2^6) = 419 (mod 481)
271^(2^8) = 16 (mod 481)
271^(321) (mod 481) = 271^(2^0 + 2^6 + 2^8) (mod 481)
= 271 * 419 * 16 (mod 481)
= 1816784(mod 481) = 47 (mod 481)


In [95]:
powers_of_two_mod(31, 629, 3551)

31^(2^0) = 31 (mod 3551)
31^(2^2) = 261 (mod 3551)
31^(2^4) = 2535 (mod 3551)
31^(2^5) = 2466 (mod 3551)
31^(2^6) = 1844 (mod 3551)
31^(2^9) = 1547 (mod 3551)
31^(629) (mod 3551) = 31 * 261 * 2535 * 2466 * 1844 * 1547 (mod 3551)
= 144286090952192280(mod 3551) = 2791 (mod 3551)


In [109]:
powers_of_two_mod(2791, 1997, 3551)

1997 = 2^0 + 2^2 + 2^3 + 2^6 + 2^7 + 2^8 + 2^9 + 2^10
2791^(2^0) = 2791 (mod 3551)
2791^(2^2) = 1255 (mod 3551)
2791^(2^3) = 1932 (mod 3551)
2791^(2^6) = 3302 (mod 3551)
2791^(2^7) = 1634 (mod 3551)
2791^(2^8) = 3155 (mod 3551)
2791^(2^9) = 572 (mod 3551)
2791^(2^10) = 492 (mod 3551)
2791^(1997) (mod 3551) = 2791^(2^0 + 2^2 + 2^3 + 2^6 + 2^7 + 2^8 + 2^9 + 2^10) (mod 3551)
= 2791 * 1255 * 1932 * 3302 * 1634 * 3155 * 572 * 492 (mod 3551)
= 5320094803028514560(mod 3551) = 2804 (mod 3551)


In [4]:
def extended_gcd(a, b, verbose=True):
    s = 0
    old_s = 1
    t = 1
    old_t = 0
    r = b
    old_r = a
    
    while r != 0:
        q = old_r // r
        old_r, r = r, old_r - q * r
        old_s, s = s, old_s - q * s
        old_t, t = t, old_t - q * t
        
    if verbose:
        print("Bézout coefficients:", (old_s % b, old_t % b))
        print("greatest common divisor:", old_r)
        print("quotients by the gcd:", (t % b, s))
    return old_s % b
    
extended_gcd(629, 3432)

Bézout coefficients: (1997, 263)
greatest common divisor: 1
quotients by the gcd: (2803, 3432)


1997

In [3]:
extended_gcd(231, 400)

Bézout coefficients: (71, 359)
greatest common divisor: 1
quotients by the gcd: (231, -400)


71

In [112]:
extended_gcd(1921, 2940)

Bézout coefficients: (2161, 509)
greatest common divisor: 1
quotients by the gcd: (1019, 2940)


In [12]:
characters = [chr(x) for x in range(ord('A'), ord('Z') + 1)]
cipher = """APHUO EGEHP PEXOV FKEUH CKVUE CHKVE APHUO
EGEHU EXOVL EXDKT VGEFT EHFKE UHCKF TZEXO
VEZDT TVKUE XOVKV ENOHK ZFTEH TEHKQ LEROF
PVEHP PEXOV ERYKP GERYT GVKEG XDRTE RGAGA"""
counts = {c: cipher.count(c) for c in characters}
counts

{'A': 4,
 'B': 0,
 'C': 3,
 'D': 3,
 'E': 26,
 'F': 6,
 'G': 8,
 'H': 12,
 'I': 0,
 'J': 0,
 'K': 12,
 'L': 2,
 'M': 0,
 'N': 1,
 'O': 9,
 'P': 8,
 'Q': 1,
 'R': 5,
 'S': 0,
 'T': 9,
 'U': 7,
 'V': 12,
 'W': 0,
 'X': 7,
 'Y': 2,
 'Z': 3}

In [6]:
extended_gcd(3, 26, False)

9

In [31]:
mono_encrypt(cipher, 5, 3, n=26)


'DAMZVXHXMAAXOVECBXZMNBEZXNMBEXDAMZVOXHXMZXOVEGXOSBUEHXCUXMCBXZMNBCUYXOVOEXYSUUEBZXOVEBEXQVMBYCUXMUXMBFGXKVCOAEXMAAXOVEXKTBAHXKTUHEBXHOSKUXKHDHD'

In [25]:
def mono_encrypt(plain, a, b, n=26):
    plain = plain.upper().replace(' ', '')
    plain = [ord(x) - ord('A') for x in plain]
    cipher = [((x * a) + b) % n for x in plain]
    cipher = [chr(x + ord('A')) for x in cipher]
    return ''.join(cipher)

def mono_decrypt(cipher, a, b, n=26):
    a_inv = extended_gcd(a, 26, False)
    b_inv = (- a_inv * b) % n
    return mono_encrypt(cipher, a_inv, b_inv)

def score(proposal):
    #characters = [chr(x) for x in range(ord('A'), ord('Z') + 1)]
#     characters = ['E', 'T', 'A', 'O', 'I']
#     return sum([proposal.count(c) for c in characters])
    return proposal.count('THE')

results = defaultdict(list)

for a in [3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25]:
    print(a)
    for b in range(1, 26):
        plain = mono_decrypt(cipher, a, b)
        results[score(plain)].append((a, b, plain))


3
5
7
9
11
15
17
19
21
23
25


In [26]:
max(results.keys())

1

In [27]:
results[1]

[(19,
  13,
  'NWMZLFBFMWWFGLKQTFZMJTKZFJMTKFNWMZLGFBFMZFGLKEFGUTOKBFQOFMQTFZMJTQOCFGLGKFCUOOKTZFGLKTKFALMTCQOFMOFMTHEFSLQGWKFMWWFGLKFSRTWBFSROBKTFBGUSOFSBNBN')]

In [21]:
s = 'ABCTHEFKXTHE'
s.count('THE')

2